In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("local").appName("midterm").getOrCreate()

In [2]:
stations_bd = spark_session.sparkContext.broadcast({'83':'Mezes Park', '84':'Ryland Park'})
stations_bd.value["83"]

'Mezes Park'

In [3]:
stationsfile = './data/stations.csv'
stationsdata = dict(map(lambda x:(x[0], x[1]), map(lambda x: x.split(','), open(stationsfile))))
stationsdata
stations_bd = spark_session.sparkContext.broadcast(stationsdata)
stations_bd.value["83"]

'Mezes Park'

In [4]:
stations_bd.unpersist()

In [5]:
status = spark_session.sparkContext.textFile("./data/status.csv")
stations = spark_session.sparkContext.textFile("./data/stations.csv")

In [6]:
status1 = status.map(lambda x: x.split(',')).keyBy(lambda x: x[0])
stations1 = stations.map(lambda x: x.split(',')).keyBy(lambda x: x[0])
status1.join(stations1) \
    .map(lambda x:(x[1][0][3], x[1][1][1], x[1][0][1], x[1][0][2])) \
    .count()

907200

In [7]:
status1.map(lambda x:(x[1][3], stationsdata[x[0]], x[1][1], x[1][2]))\
    .count()

907200

In [8]:
stations_bd = spark_session.sparkContext.broadcast(stationsdata)
stations_bd.value["83"]

'Mezes Park'

In [9]:
status1.map(lambda x:(x[1][3], stations_bd.value[x[0]], x[1][1], x[1][2]))\
    .count()

907200

In [10]:
acc=spark_session.sparkContext.accumulator(0)
def addone(x):
    global acc
    acc +=1
    return x+1
myrdd = spark_session.sparkContext.parallelize([1,2,3,4,5])
myrdd.map(lambda x: addone(x)).collect()

[2, 3, 4, 5, 6]

In [11]:
print("records processed: " + str(acc.value))

records processed: 5


In [13]:
nums = range(0, 10) # nums = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# partitionBy는 pair RDD에서만 의미
rdd = spark_session.sparkContext.parallelize(nums).map(lambda el: (el, el)) \
        .partitionBy(2) \
        .persist()
    
print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))


Number of partitions: 2
Partitioner: <pyspark.core.rdd.Partitioner object at 0x7490822216d0>
Partitions structure: [[(0, 0), (2, 2), (4, 4), (6, 6), (8, 8)], [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]


In [27]:
from pyspark.rdd import portable_hash

def country_partitioner(country):
    return portable_hash(country)

transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'James', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Hungary'},
    {'name': 'Holland', 'amount': 400, 'country': 'Hungary'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
]
rdd = spark_session.sparkContext.parallelize(transactions) \
        .map(lambda el: (el['country'], el)) \
        .partitionBy(3, country_partitioner)

print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))


Number of partitions: 3
Partitioner: <pyspark.core.rdd.Partitioner object at 0x74914c36ee40>
Partitions structure: [[('Poland', {'name': 'Marek', 'amount': 51, 'country': 'Poland'}), ('Hungary', {'name': 'Johannes', 'amount': 200, 'country': 'Hungary'}), ('Hungary', {'name': 'Holland', 'amount': 400, 'country': 'Hungary'}), ('Poland', {'name': 'Paul', 'amount': 75, 'country': 'Poland'})], [], [('United Kingdom', {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'}), ('United Kingdom', {'name': 'James', 'amount': 15, 'country': 'United Kingdom'})]]


In [28]:
re_rdd = rdd.repartition(2)
print("Number of partitions: {}".format(re_rdd.getNumPartitions()))
print("Partitioner: {}".format(re_rdd.partitioner))
print("Partitions structure: {}".format(re_rdd.glom().collect()))

Number of partitions: 2
Partitioner: None
Partitions structure: [[('United Kingdom', {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'}), ('United Kingdom', {'name': 'James', 'amount': 15, 'country': 'United Kingdom'})], [('Poland', {'name': 'Marek', 'amount': 51, 'country': 'Poland'}), ('Hungary', {'name': 'Johannes', 'amount': 200, 'country': 'Hungary'}), ('Hungary', {'name': 'Holland', 'amount': 400, 'country': 'Hungary'}), ('Poland', {'name': 'Paul', 'amount': 75, 'country': 'Poland'})]]
